In [40]:
import os
from pyspark.sql.types import *
from pyspark.sql import functions as F
%load_ext autoreload
%autoreload 2
import pandas as pd
import pyspark
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import uv_model
import matplotlib.pyplot as plt
#import spark_notebook_helpers
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import numpy as np


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
venues = pd.read_csv('venues_clean.csv')
events = pd.read_csv('events_clean.csv')
artists = pd.read_csv('artists.csv')

Start a spark

In [8]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [9]:
matrix_raw = events[['artists_id', 'venue_id']].copy()
matrix_raw['event_counts'] = 1
matrix_raw = matrix_raw.groupby(['artists_id', 'venue_id'],\
                                as_index = False).count()
ratings_matrix = pd.DataFrame(matrix_raw)

Check to see what the big bookers are

In [10]:
bigbookers = list(ratings_matrix[ratings_matrix['event_counts'] >= 20]['venue_id'])

In [11]:
venues[venues['venue_id'].isin(bigbookers)]

,Unnamed: 0,venueidentifer,venue_name,country,region,city,genre,bio,latitude,longitude,venue_id
174,174,ahmanson theatre Los Angeles CA United States,ahmanson theatre,United States,CA,Los Angeles,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",34.056607,-118.248932,174
536,536,barboza Seattle WA United States,barboza,United States,WA,Seattle,"['Pop, Indie Pop', 'Indie Rock', 'Leeds', 'Pos...","['New album ""Lightning Round"" out August 3rd v...",47.612338,-122.322064,536
628,628,belly room West Hollywood CA United States,belly room,United States,CA,West Hollywood,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",34.090000,-118.360833,628
982,982,budweiser brewery experience Fairfield CA Unit...,budweiser brewery experience,United States,CA,Fairfield,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",38.236427,-122.093740,982
1017,1017,cafe San Francisco CA United States,cafe,United States,CA,San Francisco,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",37.764154,-122.448191,1017
1489,1489,condor club San Francisco CA United States,condor club,United States,CA,San Francisco,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Rodney Lee Castillo and Midnight train ,Now ...",37.798008,-122.406653,1489
1716,1716,davies symphony hall San Francisco CA United S...,davies symphony hall,United States,CA,San Francisco,"['Pop, Jazz, Dance Music', 'Pop, Jazz, Dance M...",['“Pink Martini is a rollicking around-the-wor...,37.777615,-122.420614,1716
1719,1719,dawsons jam Tacoma WA United States,dawsons jam,United States,WA,Tacoma,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",47.253056,-122.443056,1719
1826,1826,disney concert hall Los Angeles CA United States,disney concert hall,United States,CA,Los Angeles,"['Classical', 'Classical', 'Pop, Jazz, Dance M...","[nan, nan, '“Pink Martini is a rollicking arou...",34.054754,-118.249426,1826
2638,2638,golden gate theater San Francisco CA United St...,golden gate theater,United States,CA,San Francisco,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",37.782571,-122.411074,2638


In [14]:
ratings = spark.createDataFrame(ratings_matrix)

In [16]:
ratings_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24236 entries, 0 to 24235
Data columns (total 3 columns):
artists_id      24236 non-null int64
venue_id        24236 non-null int64
event_counts    24236 non-null int64
dtypes: int64(3)
memory usage: 757.4 KB


In [21]:
(split_60_df, split_a_20_df, split_b_20_df) = ratings.randomSplit([0.6, 0.2, 0.2], seed = 1997)

In [22]:
training_df = split_60_df.cache()
validation_df = split_a_20_df.cache()
test_df = split_b_20_df.cache()

In [23]:
print('Training: {0}, validation: {1}, test: {2}\n'.format(
  training_df.count(), validation_df.count(), test_df.count())
)
training_df.show(3)
validation_df.show(3)
test_df.show(3)

Training: 14640, validation: 4766, test: 4830

+----------+--------+------------+
|artists_id|venue_id|event_counts|
+----------+--------+------------+
|         7|    2995|           1|
|        12|    2347|           1|
|        12|    7233|           1|
+----------+--------+------------+
only showing top 3 rows

+----------+--------+------------+
|artists_id|venue_id|event_counts|
+----------+--------+------------+
|         4|     327|           1|
|        12|    4109|           1|
|        12|    4588|           1|
+----------+--------+------------+
only showing top 3 rows

+----------+--------+------------+
|artists_id|venue_id|event_counts|
+----------+--------+------------+
|         7|    5879|           1|
|        12|    6870|           1|
|        14|    4780|           1|
+----------+--------+------------+
only showing top 3 rows



In [24]:
validation_df = validation_df.withColumn("event_counts", validation_df["event_counts"].cast(DoubleType()))

In [26]:
als = ALS()

In [ ]:
grid search several peramaters

First Go Using number of plays

In [34]:
als.setMaxIter(5)\
   .setSeed(1996)\
   .setItemCol("venue_id")\
   .setRatingCol("event_counts")\
   .setUserCol("artists_id")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="event_counts", metricName="rmse")

tolerance = 0.03
ranks = [4, 8, 12, 16]
regParams = [0.15, 0.2, 0.25]
errors = [[0]*len(ranks)]*len(regParams)
models = [[0]*len(ranks)]*len(regParams)
err = 0
min_error = float('inf')
best_rank = -1
i = 0
for regParam in regParams:
  j = 0
  for rank in ranks:
    # Set the rank here:
    als.setParams(rank = rank, regParam = regParam, implicitPrefs=True, nonnegative=True)
    # Create the model with these parameters.
    model = als.fit(training_df)
    # Run the model to create a prediction. Predict against the validation_df.
    predict_df = model.transform(validation_df)

    # Remove NaN values from prediction (due to SPARK-14489)
    predicted_plays_df = predict_df.filter(predict_df.prediction != float('nan'))
    predicted_plays_df = predicted_plays_df.withColumn("prediction", F.abs(F.round(predicted_plays_df["prediction"],0)))
    
    # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
    error = reg_eval.evaluate(predicted_plays_df)
    errors[i][j] = error
    models[i][j] = model
    print ('For rank %s, regularization parameter %s the RMSE is %s' % (rank, regParam, error))
    if error < min_error:
      min_error = error
      best_params = [i,j]
    j += 1
  i += 1

als.setRegParam(regParams[best_params[0]])
als.setRank(ranks[best_params[1]])
print ('The best model was trained with regularization parameter %s' % regParams[best_params[0]])
print ('The best model was trained with rank %s' % ranks[best_params[1]])
my_model = models[best_params[0]][best_params[1]]

For rank 4, regularization parameter 0.15 the RMSE is 1.9322107767540313
For rank 8, regularization parameter 0.15 the RMSE is 1.9322107767540313
For rank 12, regularization parameter 0.15 the RMSE is 1.932108735045172
For rank 16, regularization parameter 0.15 the RMSE is 1.9319046354581615
For rank 4, regularization parameter 0.2 the RMSE is 1.9322107767540313
For rank 8, regularization parameter 0.2 the RMSE is 1.9322107767540313
For rank 12, regularization parameter 0.2 the RMSE is 1.9322107767540313
For rank 16, regularization parameter 0.2 the RMSE is 1.9319046354581615
For rank 4, regularization parameter 0.25 the RMSE is 1.9322107767540313
For rank 8, regularization parameter 0.25 the RMSE is 1.9322107767540313
For rank 12, regularization parameter 0.25 the RMSE is 1.9322107767540313
For rank 16, regularization parameter 0.25 the RMSE is 1.9322107767540313
The best model was trained with regularization parameter 0.15
The best model was trained with rank 16


1.931904
The best result is reg = .15 rank = 16
Try is again with smaller reg perameters and larger rank

In [35]:
als.setMaxIter(5)\
   .setSeed(1996)\
   .setItemCol("venue_id")\
   .setRatingCol("event_counts")\
   .setUserCol("artists_id")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="exp_prediction", labelCol="event_counts", metricName="rmse")

tolerance = 0.03
ranks = [16,18,20]
regParams = [0.1, 0.13, 0.15]
errors = [[0]*len(ranks)]*len(regParams)
models = [[0]*len(ranks)]*len(regParams)
err = 0
min_error = float('inf')
best_rank = -1
i = 0
for regParam in regParams:
  j = 0
  for rank in ranks:
    # Set the rank here:
    als.setParams(rank = rank, regParam = regParam, nonnegative=True, implicitPrefs=True)
    # Create the model with these parameters.
    model = als.fit(training_df)
    # Run the model to create a prediction. Predict against the validation_df.
    predict_df = model.transform(validation_df)

    # Remove NaN values from prediction (due to SPARK-14489)
    predicted_plays_df = predict_df.filter(predict_df.prediction != float('nan'))
    predicted_plays_df = predicted_plays_df.withColumn("exp_prediction", F.abs(F.round(predicted_plays_df["prediction"],0)))
    # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
    error = reg_eval.evaluate(predicted_plays_df)
    errors[i][j] = error
    models[i][j] = model
    print ('For rank %s, regularization parameter %s the RMSE is %s' % (rank, regParam, error))
    if error < min_error:
      min_error = error
      best_params = [i,j]
    j += 1
  i += 1

als.setRegParam(regParams[best_params[0]])
als.setRank(ranks[best_params[1]])
print ('The best model was trained with regularization parameter %s' % regParams[best_params[0]])
print ('The best model was trained with rank %s' % ranks[best_params[1]])
my_model = models[best_params[0]][best_params[1]]

For rank 16, regularization parameter 0.1 the RMSE is 1.9317005143063999
For rank 18, regularization parameter 0.1 the RMSE is 1.9319046354581615
For rank 20, regularization parameter 0.1 the RMSE is 1.9322107767540313
For rank 16, regularization parameter 0.13 the RMSE is 1.9319046354581615
For rank 18, regularization parameter 0.13 the RMSE is 1.9319046354581615
For rank 20, regularization parameter 0.13 the RMSE is 1.9322107767540313
For rank 16, regularization parameter 0.15 the RMSE is 1.9319046354581615
For rank 18, regularization parameter 0.15 the RMSE is 1.9319046354581615
For rank 20, regularization parameter 0.15 the RMSE is 1.9322107767540313
The best model was trained with regularization parameter 0.1
The best model was trained with rank 16


1.93170
seems like reg perameter around .1 and a rank around 16 is best
lets try is with reg perameters .05 to .1

In [36]:
als.setMaxIter(5)\
   .setSeed(1996)\
   .setItemCol("venue_id")\
   .setRatingCol("event_counts")\
   .setUserCol("artists_id")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="event_counts", metricName="rmse")

tolerance = 0.03
ranks = [15,16,17]
regParams = [0.05, 0.075, 0.15]
errors = [[0]*len(ranks)]*len(regParams)
models = [[0]*len(ranks)]*len(regParams)
err = 0
min_error = float('inf')
best_rank = -1
i = 0
for regParam in regParams:
  j = 0
  for rank in ranks:
    # Set the rank here:
    als.setParams(rank = rank, regParam = regParam, nonnegative=True, implicitPrefs=True)
    # Create the model with these parameters.
    model = als.fit(training_df)
    # Run the model to create a prediction. Predict against the validation_df.
    predict_df = model.transform(validation_df)

    # Remove NaN values from prediction (due to SPARK-14489)
    predicted_plays_df = predict_df.filter(predict_df.prediction != float('nan'))
    predicted_plays_df = predicted_plays_df.withColumn("prediction", F.abs(F.round(predicted_plays_df["prediction"],0)))
    # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
    error = reg_eval.evaluate(predicted_plays_df)
    errors[i][j] = error
    models[i][j] = model
    print ('For rank %s, regularization parameter %s the RMSE is %s' % (rank, regParam, error))
    if error < min_error:
      min_error = error
      best_params = [i,j]
    j += 1
  i += 1

als.setRegParam(regParams[best_params[0]])
als.setRank(ranks[best_params[1]])
print ('The best model was trained with regularization parameter %s' % regParams[best_params[0]])
print ('The best model was trained with rank %s' % ranks[best_params[1]])
my_model = models[best_params[0]][best_params[1]]

For rank 15, regularization parameter 0.05 the RMSE is 1.9320066879468336
For rank 16, regularization parameter 0.05 the RMSE is 1.9317005143063999
For rank 17, regularization parameter 0.05 the RMSE is 1.9312922072812708
For rank 15, regularization parameter 0.075 the RMSE is 1.9320066879468336
For rank 16, regularization parameter 0.075 the RMSE is 1.9315984456416007
For rank 17, regularization parameter 0.075 the RMSE is 1.9313942921298912
For rank 15, regularization parameter 0.15 the RMSE is 1.932108735045172
For rank 16, regularization parameter 0.15 the RMSE is 1.9319046354581615
For rank 17, regularization parameter 0.15 the RMSE is 1.9314963715830495
The best model was trained with regularization parameter 0.05
The best model was trained with rank 17


1.93129
a little bit better

In [117]:
als.setMaxIter(5)\
   .setSeed(1996)\
   .setItemCol("venue_id")\
   .setRatingCol("event_counts")\
   .setUserCol("artists_id")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="exp_rediction", labelCol="event_counts", metricName="rmse")

tolerance = 0.03
ranks = [16,17,18]
regParams = [0.01, 0.03, 0.05]
errors = [[0]*len(ranks)]*len(regParams)
models = [[0]*len(ranks)]*len(regParams)
err = 0
min_error = float('inf')
best_rank = -1
i = 0
for regParam in regParams:
  j = 0
  for rank in ranks:
    # Set the rank here:
    als.setParams(rank = rank, regParam = regParam, nonnegative=True, implicitPrefs=True)
    # Create the model with these parameters.
    model = als.fit(training_df)
    # Run the model to create a prediction. Predict against the validation_df.
    predict_df = model.transform(validation_df)

    # Remove NaN values from prediction (due to SPARK-14489)
    predicted_plays_df = predict_df.filter(predict_df.prediction != float('nan'))
    predicted_plays_df = predicted_plays_df.withColumn("exp_prediction", F.abs(F.round(predicted_plays_df["prediction"],0)))
    # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
    error = reg_eval.evaluate(predicted_plays_df)
    errors[i][j] = error
    models[i][j] = model
    print ('For rank %s, regularization parameter %s the RMSE is %s' % (rank, regParam, error))
    if error < min_error:
      min_error = error
      best_params = [i,j]
    j += 1
  i += 1

als.setRegParam(regParams[best_params[0]])
als.setRank(ranks[best_params[1]])
print ('The best model was trained with regularization parameter %s' % regParams[best_params[0]])
print ('The best model was trained with rank %s' % ranks[best_params[1]])
my_model = models[best_params[0]][best_params[1]]

For rank 16, regularization parameter 0.01 the RMSE is 1.9313942921298912
For rank 17, regularization parameter 0.01 the RMSE is 1.9315984456416007
For rank 18, regularization parameter 0.01 the RMSE is 1.9318025775783016
For rank 16, regularization parameter 0.03 the RMSE is 1.9314963715830495
For rank 17, regularization parameter 0.03 the RMSE is 1.9311901170363317
For rank 18, regularization parameter 0.03 the RMSE is 1.9319046354581615
For rank 16, regularization parameter 0.05 the RMSE is 1.9317005143063999
For rank 17, regularization parameter 0.05 the RMSE is 1.9312922072812708
For rank 18, regularization parameter 0.05 the RMSE is 1.9319046354581615
The best model was trained with regularization parameter 0.03
The best model was trained with rank 17


1.93119 is the best so far with rank 17 and reg.03
lets try the same thing but lets fit and train on the log of the number of events insted.

In [72]:
als.setMaxIter(5)\
   .setSeed(1996)\
   .setItemCol("venue_id")\
   .setRatingCol("log_play")\
   .setUserCol("artists_id")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="exp_prediction", labelCol="event_counts", metricName="rmse")

tolerance = 0.03
ranks = [10, 15, 20]
regParams = [0.01, 0.03, 0.05, .10]
errors = [[0]*len(ranks)]*len(regParams)
models = [[0]*len(ranks)]*len(regParams)
err = 0
min_error = float('inf')
best_rank = -1
i = 0
for regParam in regParams:
  j = 0
  for rank in ranks:
    # Set the rank here:
    als.setParams(rank = rank, regParam = regParam, nonnegative=True, implicitPrefs=True)
    # Create the model with these parameters.
    training_df_log = training_df.withColumn("log_play", log1p("event_counts"))
    model = als.fit(training_df_log)
    # Run the model to create a prediction. Predict against the validation_df.
    validation_df_log = validation_df.withColumn("log_play", log1p("event_counts"))
    predict_df = model.transform(validation_df)

    # Remove NaN values from prediction (due to SPARK-14489)
    predicted_plays_df = predict_df.filter(predict_df.prediction != float('nan'))
    predicted_plays_df = predicted_plays_df.withColumn('exp_predictied', expm1('prediction'))
    predicted_plays_df = predicted_plays_df.withColumn("exp_prediction", F.abs(F.round(predicted_plays_df["prediction"],0)))
    # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
    error = reg_eval.evaluate(predicted_plays_df)
    errors[i][j] = error
    models[i][j] = model
    print ('For rank %s, regularization parameter %s the RMSE is %s' % (rank, regParam, error))
    if error < min_error:
      min_error = error
      best_params = [i,j]
    j += 1
  i += 1

als.setRegParam(regParams[best_params[0]])
als.setRank(ranks[best_params[1]])
print ('The best model was trained with regularization parameter %s' % regParams[best_params[0]])
print ('The best model was trained with rank %s' % ranks[best_params[1]])
my_model = models[best_params[0]][best_params[1]]

For rank 10, regularization parameter 0.01 the RMSE is 1.9320066879468336
For rank 15, regularization parameter 0.01 the RMSE is 1.9315984456416007
For rank 20, regularization parameter 0.01 the RMSE is 1.9314963715830495
For rank 10, regularization parameter 0.03 the RMSE is 1.9320066879468336
For rank 15, regularization parameter 0.03 the RMSE is 1.9317005143063999
For rank 20, regularization parameter 0.03 the RMSE is 1.9311901170363317
For rank 10, regularization parameter 0.05 the RMSE is 1.9320066879468336
For rank 15, regularization parameter 0.05 the RMSE is 1.9317005143063999
For rank 20, regularization parameter 0.05 the RMSE is 1.9313942921298912
For rank 10, regularization parameter 0.1 the RMSE is 1.9319046354581615
For rank 15, regularization parameter 0.1 the RMSE is 1.9317005143063999
For rank 20, regularization parameter 0.1 the RMSE is 1.9317005143063999
The best model was trained with regularization parameter 0.03
The best model was trained with rank 20


1.93119 matches the best so far
Lets try increasing rand and keeping the reg aroung .03

In [73]:
als.setMaxIter(5)\
   .setSeed(1996)\
   .setItemCol("venue_id")\
   .setRatingCol("log_play")\
   .setUserCol("artists_id")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="exp_prediction", labelCol="event_counts", metricName="rmse")

tolerance = 0.03
ranks = [18, 20, 22, 24]
regParams = [0.01, 0.03, 0.05]
errors = [[0]*len(ranks)]*len(regParams)
models = [[0]*len(ranks)]*len(regParams)
err = 0
min_error = float('inf')
best_rank = -1
i = 0
for regParam in regParams:
  j = 0
  for rank in ranks:
    # Set the rank here:
    als.setParams(rank = rank, regParam = regParam, nonnegative=True, implicitPrefs=True)
    # Create the model with these parameters.
    training_df_log = training_df.withColumn("log_play", log1p("event_counts"))
    model = als.fit(training_df_log)
    # Run the model to create a prediction. Predict against the validation_df.
    validation_df_log = validation_df.withColumn("log_play", log1p("event_counts"))
    predict_df = model.transform(validation_df)

    # Remove NaN values from prediction (due to SPARK-14489)
    predicted_plays_df = predict_df.filter(predict_df.prediction != float('nan'))
    predicted_plays_df = predicted_plays_df.withColumn('exp_predictied', expm1('prediction'))
    predicted_plays_df = predicted_plays_df.withColumn("exp_prediction", F.abs(F.round(predicted_plays_df["prediction"],0)))
    # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
    error = reg_eval.evaluate(predicted_plays_df)
    errors[i][j] = error
    models[i][j] = model
    print ('For rank %s, regularization parameter %s the RMSE is %s' % (rank, regParam, error))
    if error < min_error:
      min_error = error
      best_params = [i,j]
    j += 1
  i += 1

als.setRegParam(regParams[best_params[0]])
als.setRank(ranks[best_params[1]])
print ('The best model was trained with regularization parameter %s' % regParams[best_params[0]])
print ('The best model was trained with rank %s' % ranks[best_params[1]])
my_model = models[best_params[0]][best_params[1]]

For rank 18, regularization parameter 0.01 the RMSE is 1.9313942921298912
For rank 20, regularization parameter 0.01 the RMSE is 1.9314963715830495
For rank 22, regularization parameter 0.01 the RMSE is 1.9313942921298912
For rank 24, regularization parameter 0.01 the RMSE is 1.9320066879468336
For rank 18, regularization parameter 0.03 the RMSE is 1.9313942921298912
For rank 20, regularization parameter 0.03 the RMSE is 1.9311901170363317
For rank 22, regularization parameter 0.03 the RMSE is 1.9313942921298912
For rank 24, regularization parameter 0.03 the RMSE is 1.932108735045172
For rank 18, regularization parameter 0.05 the RMSE is 1.9313942921298912
For rank 20, regularization parameter 0.05 the RMSE is 1.9313942921298912
For rank 22, regularization parameter 0.05 the RMSE is 1.9312922072812708
For rank 24, regularization parameter 0.05 the RMSE is 1.932108735045172
The best model was trained with regularization parameter 0.03
The best model was trained with rank 20


Same result as the so lets not convert the target data to the log
The best model had 1.93119 with rank 17 and reg.03
Now lets see how we did on the test data that was split in the beginning

fist lets start over and fit trian the model with these perams 

In [106]:
model = ALS(maxIter = 5,
   seed = 1996,
   itemCol = "venue_id",
   userCol = 'artists_id',
   ratingCol = "event_counts",
   rank = 17, 
   regParam = .03,
   nonnegative=True,
   implicitPrefs=True)

In [108]:
best_model = model.fit(training_df)

In [113]:
predictions_df = best_model.transform(test_df)

In [118]:
predictions.show()

+----------+--------+------------+------------+
|artists_id|venue_id|event_counts|  prediction|
+----------+--------+------------+------------+
|    470447|    1088|         1.0|2.1346827E-4|
|     50621|    1580|         1.0|         NaN|
|   2430005|    1591|         1.0|         NaN|
|   3491873|    2659|         1.0|0.0020782612|
|    326584|    2659|         1.0|3.0284064E-4|
|  12006479|    2659|         1.0| 5.220097E-6|
|  12285401|    4101|         1.0|         NaN|
|  11169720|    4519|         1.0|         NaN|
|    327482|    4818|         1.0|         NaN|
|    158271|    5518|         1.0|         NaN|
|   1953871|    6654|         1.0|         NaN|
|   7503101|    7240|         1.0|         NaN|
|   1945144|    7240|         1.0|1.0569948E-4|
|   1683974|     623|         1.0|         NaN|
|  11553900|     737|         1.0|         NaN|
|   1211969|     897|         1.0|         NaN|
|   4519445|    1483|         1.0|         NaN|
|    283444|    1896|         1.0|      

In [119]:
reg_eval = RegressionEvaluator(predictionCol="exp_prediction", labelCol="event_counts", metricName="rmse")
# Remove NaN values from prediction (due to SPARK-14489)
predicted_test_df = predict_df.filter(predict_df.prediction != float('nan'))

# Round floats to whole numbers
predicted_test_df = predicted_test_df.withColumn("exp_prediction", F.abs(F.round(predicted_test_df["prediction"],0)))
# Run the previously created RMSE evaluator, reg_eval, on the predicted_test_df DataFrame
test_RMSE = reg_eval.evaluate(predicted_test_df)

print('The model had a RMSE on the test set of {0}'.format(test_RMSE))

The model had a RMSE on the test set of 1.9319046354581615


thats not bad lets go with that